In [1]:
import os
import pathlib

import tensorflow as tf
import tensorflow_io as tfio


from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from keras_flops import get_flops

from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

import math
!set XLA_FLAGS=--xla_gpu_cuda_data_dir="C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.8"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

C:\Users\Stan\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Num GPUs Available:  1


In [2]:
# DATA
BATCH_SIZE = 64
AUTO = tf.data.AUTOTUNE
#INPUT_SHAPE = (32, 32, 3)
INPUT_SHAPE = (124, 129, 1)
NUM_CLASSES = 8

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-4

# TRAINING
EPOCHS = 20

# AUGMENTATION
IMAGE_SIZE = 64  # We will resize input images to this size.
PATCH_SIZE = 8  # Size of the patches to be extracted from the input images.
NUM_PATCHES = (IMAGE_SIZE // PATCH_SIZE) ** 2

# ViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8
MLP_UNITS = [
    PROJECTION_DIM * 2,
    PROJECTION_DIM,
]

# TOKENLEARNER
NUM_TOKENS = 8

# Data Processing

In [3]:
DATASET_PATH = 'data/mini_speech_commands'

data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

In [4]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[(commands != 'README.md') & (commands != '.DS_Store')]
print('Commands:', commands)

Commands: ['down' 'go' 'left' 'no' 'right' 'stop' 'up' 'yes']


In [5]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    seed=0,
    output_sequence_length=16000,
    subset='both')

label_names = np.array(train_ds.class_names)
print()
print("label names:", label_names)

Found 8000 files belonging to 8 classes.
Using 6400 files for training.
Using 1600 files for validation.

label names: ['down' 'go' 'left' 'no' 'right' 'stop' 'up' 'yes']


In [6]:
def squeeze(audio, labels):
  audio = tf.squeeze(audio, axis=-1)
  return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

In [7]:
test_ds = val_ds.shard(num_shards=2, index=0)
val_ds = val_ds.shard(num_shards=2, index=1)

In [8]:
for example_audio, example_labels in train_ds.take(1):  
  print(example_audio.shape)
  print(example_labels.shape)

(64, 16000)
(64,)


In [10]:
def get_spectrogram(waveform):
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram

def make_spec_ds(ds):
  return ds.map(
      map_func=lambda audio,label: (get_spectrogram(audio), label),
      num_parallel_calls=tf.data.AUTOTUNE)


def get_spectrogram(waveform, training=False):
  # Convert the waveform to a spectrogram via a STFT.
    spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
    if training:
        spectrogram = tfio.audio.freq_mask(spectrogram, param=10)
        spectrogram = tfio.audio.time_mask(spectrogram, param=10)
    # Obtain the magnitude of the STFT.
    spectrogram = tf.abs(spectrogram)
    # Add a `channels` dimension, so that the spectrogram can be used
    # as image-like input data with convolution layers (which expect
    # shape (`batch_size`, `height`, `width`, `channels`).
    spectrogram = spectrogram[..., tf.newaxis]
    return spectrogram

def make_spec_ds(ds, training = False):
    if training:
        return ds.map(
          map_func=lambda audio,label: (get_spectrogram(audio), label),
          num_parallel_calls=tf.data.AUTOTUNE)
    else:
        return ds.map(
              map_func=lambda audio,label: (get_spectrogram(audio), label),
              num_parallel_calls=tf.data.AUTOTUNE)

In [11]:
train_spectrogram_ds = make_spec_ds(train_ds)
val_spectrogram_ds = make_spec_ds(val_ds)
test_spectrogram_ds = make_spec_ds(test_ds)

In [12]:
train_spectrogram_ds = train_spectrogram_ds.cache().shuffle(10000).prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = val_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = test_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)

In [13]:
norm_layer = layers.Normalization()
norm_layer.adapt(data=train_spectrogram_ds.map(map_func=lambda spec, label: spec))

data_augmentation = keras.Sequential(
    [
        layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
        norm_layer,
    ],
    name="data_augmentation",
)

# Token Learner

In [14]:
def position_embedding(
    projected_patches, num_patches=NUM_PATCHES, projection_dim=PROJECTION_DIM
):
    # Build the positions.
    positions = tf.range(start=0, limit=num_patches, delta=1)

    # Encode the positions with an Embedding layer.
    encoded_positions = layers.Embedding(
        input_dim=num_patches, output_dim=projection_dim
    )(positions)

    # Add encoded positions to the projected patches.
    return projected_patches + encoded_positions

In [15]:
def mlp(x, dropout_rate, hidden_units):
    # Iterate over the hidden units and
    # add Dense => Dropout.
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [16]:
def token_learner(inputs, number_of_tokens=NUM_TOKENS):
    # Layer normalize the inputs.
    x = layers.LayerNormalization(epsilon=LAYER_NORM_EPS)(inputs)  # (B, H, W, C)

    # Applying Conv2D => Reshape => Permute
    # The reshape and permute is done to help with the next steps of
    # multiplication and Global Average Pooling.
    attention_maps = keras.Sequential(
        [
            # 3 layers of conv with gelu activation as suggested
            # in the paper.
            layers.Conv2D(
                filters=number_of_tokens,
                kernel_size=(3, 3),
                activation=tf.nn.gelu,
                padding="same",
                use_bias=False,
            ),
            layers.Conv2D(
                filters=number_of_tokens,
                kernel_size=(3, 3),
                activation=tf.nn.gelu,
                padding="same",
                use_bias=False,
            ),
            layers.Conv2D(
                filters=number_of_tokens,
                kernel_size=(3, 3),
                activation=tf.nn.gelu,
                padding="same",
                use_bias=False,
            ),
            # This conv layer will generate the attention maps
            layers.Conv2D(
                filters=number_of_tokens,
                kernel_size=(3, 3),
                activation="sigmoid",  # Note sigmoid for [0, 1] output
                padding="same",
                use_bias=False,
            ),
            # Reshape and Permute
            layers.Reshape((-1, number_of_tokens)),  # (B, H*W, num_of_tokens)
            layers.Permute((2, 1)),
        ]
    )(
        x
    )  # (B, num_of_tokens, H*W)

    # Reshape the input to align it with the output of the conv block.
    num_filters = inputs.shape[-1]
    inputs = layers.Reshape((1, -1, num_filters))(inputs)  # inputs == (B, 1, H*W, C)

    # Element-Wise multiplication of the attention maps and the inputs
    attended_inputs = (
        attention_maps[..., tf.newaxis] * inputs
    )  # (B, num_tokens, H*W, C)

    # Global average pooling the element wise multiplication result.
    outputs = tf.reduce_mean(attended_inputs, axis=2)  # (B, num_tokens, C)
    return outputs

In [17]:
def transformer(encoded_patches):
    # Layer normalization 1.
    x1 = layers.LayerNormalization(epsilon=LAYER_NORM_EPS)(encoded_patches)

    # Multi Head Self Attention layer 1.
    attention_output = layers.MultiHeadAttention(
        num_heads=NUM_HEADS, key_dim=PROJECTION_DIM, dropout=0.1
    )(x1, x1)

    # Skip connection 1.
    x2 = layers.Add()([attention_output, encoded_patches])

    # Layer normalization 2.
    x3 = layers.LayerNormalization(epsilon=LAYER_NORM_EPS)(x2)

    # MLP layer 1.
    x4 = mlp(x3, hidden_units=MLP_UNITS, dropout_rate=0.1)

    # Skip connection 2.
    encoded_patches = layers.Add()([x4, x2])
    return encoded_patches

In [18]:
def bottleneck_block(x, expand=64, squeeze=16):

  m = keras.Sequential([
      layers.Conv2D(expand, (1,1), padding='same'),
      layers.BatchNormalization(),
      layers.Activation('relu6'),
      layers.DepthwiseConv2D((3,3), padding='same'),
      layers.BatchNormalization(),
      layers.Activation('relu6'),
      layers.Conv2D(squeeze, (1,1), padding='same'),
      layers.BatchNormalization()
  ])(x)
  return layers.Add()([m, x])

In [19]:
def create_vit_classifier(use_token_learner=True, token_learner_units=NUM_TOKENS):
    inputs = layers.Input(shape=INPUT_SHAPE)  # (B, H, W, C)
    
    # Augment data.
    augmented = data_augmentation(inputs)

    mbconv = bottleneck_block(augmented)
    #mbconv = bottleneck_block(mbconv)
    
    # Create patches and project the pathces.
    projected_patches = layers.Conv2D(
        filters=PROJECTION_DIM,
        kernel_size=(PATCH_SIZE, PATCH_SIZE),
        strides=(PATCH_SIZE, PATCH_SIZE),
        padding="VALID",
    )(mbconv)
    
    if use_token_learner:
            projected_patches = token_learner(
                projected_patches, token_learner_units
            )
            # _, hh, c = projected_patches.shape
            # h = int(math.sqrt(hh))
            # projected_patches = layers.Reshape((h, h, c))(
            #     projected_patches)
            # _, h, w, c = projected_patches.shape
            # print(h,w,c)
            # projected_patches = layers.Reshape((h * w, c))(
            #     projected_patches
            # )  # (B, number_patches, projection_dim)

            # Add positional embeddings to the projected patches.
            encoded_patches = position_embedding(
                projected_patches, num_patches = NUM_TOKENS
            )  # (B, number_patches, projection_dim)
            encoded_patches = layers.Dropout(0.1)(encoded_patches)
    else:
        _, h, w, c = projected_patches.shape
        print(h,w,c)
        projected_patches = layers.Reshape((h * w, c))(
            projected_patches
        )  # (B, number_patches, projection_dim)

        # Add positional embeddings to the projected patches.
        encoded_patches = position_embedding(
            projected_patches
        )  # (B, number_patches, projection_dim)
        encoded_patches = layers.Dropout(0.1)(encoded_patches)

    # Iterate over the number of layers and stack up blocks of
    # Transformer.
    for i in range(NUM_LAYERS):
        # Add a Transformer block.
        encoded_patches = transformer(encoded_patches)

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=LAYER_NORM_EPS)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [20]:
def run_experiment(model):
    # Initialize the AdamW optimizer.
    optimizer = tfa.optimizers.AdamW(
        learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(3, name="top-3-accuracy"),
        ],
    )

    # Define callbacks
    checkpoint_filepath = "./tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    # Custom Scheduler Function
    lr_start   = 1e-4
    lr_max     = 0.000015 * BATCH_SIZE
    lr_min     = 1e-7
    lr_ramp_ep = 3
    lr_sus_ep  = 0
    lr_decay   = 0.7
      
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
                
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
                
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
                
        return lr


    # Using this Custom Function, create a Callback
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)


    # Train the model.
    _ = model.fit(
        train_spectrogram_ds,
        epochs=EPOCHS,
        validation_data=val_spectrogram_ds,
        callbacks=[lr_callback, checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_3_accuracy = model.evaluate(test_spectrogram_ds)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 3 accuracy: {round(top_3_accuracy * 100, 2)}%")


In [21]:
model = create_vit_classifier(use_token_learner=True)
print(get_flops(model, batch_size=1))
model = create_vit_classifier(use_token_learner=False)
print(get_flops(model, batch_size=1))

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
42852841
8 8 128
105232465


In [22]:
vit_token_learner = create_vit_classifier()
run_experiment(vit_token_learner)


Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/20
100/100 [==============================] - 22s 126ms/step - loss: 1.8876 - accuracy: 0.2881 - top-3-accuracy: 0.6152 - val_loss: 3.0323 - val_accuracy: 0.1562 - val_top-3-accuracy: 0.4167 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.00038666666666666667.
Epoch 2/20
100/100 [==============================] - 11s 108ms/step - loss: 1.5687 - accuracy: 0.4191 - top-3-accuracy: 0.7653 - val_loss: 3.8437 - val_accuracy: 0.1445 - val_top-3-accuracy: 0.3659 - lr: 3.8667e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0006733333333333334.
Epoch 3/20
100/100 [==============================] - 11s 113ms/step - loss: 1.1696 - accuracy: 0.5594 - top-3-accuracy: 0.8737 - val_loss: 4.0991 - val_accuracy: 0.1758 - val_top-3-accuracy: 0.3646 - lr: 6.7333e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.00096.
Epoch 4/20
100/100 [==============================] - 11s 11

In [23]:
vit_token_learner = create_vit_classifier(use_token_learner=False)
run_experiment(vit_token_learner)

8 8 128

Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/20
100/100 [==============================] - 33s 250ms/step - loss: 2.0047 - accuracy: 0.2259 - top-3-accuracy: 0.5350 - val_loss: 3.5932 - val_accuracy: 0.1042 - val_top-3-accuracy: 0.3737 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.00038666666666666667.
Epoch 2/20
100/100 [==============================] - 24s 239ms/step - loss: 1.4865 - accuracy: 0.4400 - top-3-accuracy: 0.7822 - val_loss: 2.5731 - val_accuracy: 0.2214 - val_top-3-accuracy: 0.5195 - lr: 3.8667e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0006733333333333334.
Epoch 3/20
100/100 [==============================] - 24s 238ms/step - loss: 0.9339 - accuracy: 0.6617 - top-3-accuracy: 0.9131 - val_loss: 2.5640 - val_accuracy: 0.2812 - val_top-3-accuracy: 0.6250 - lr: 6.7333e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.00096.
Epoch 4/20
100/100 [==============================] 

In [24]:
# import tensorflow as tf
# from tensorflow.python.profiler.model_analyzer import profile
# from tensorflow.python.profiler.option_builder import ProfileOptionBuilder
# print('TensorFlow:', tf.__version__)

# model = create_vit_classifier(use_token_learner=True)

# forward_pass = tf.function(
#     model.call,
#     input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])

# graph_info = profile(forward_pass.get_concrete_function().graph,
#                         options=ProfileOptionBuilder.float_operation())

# # The //2 is necessary since `profile` counts multiply and accumulate
# # as two flops, here we report the total number of multiply accumulate ops
# flops = graph_info.total_float_ops // 2
# print('Flops: {:,}'.format(flops))

# model = create_vit_classifier(use_token_learner=False)

# forward_pass = tf.function(
#     model.call,
#     input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])

# graph_info = profile(forward_pass.get_concrete_function().graph,
#                         options=ProfileOptionBuilder.float_operation())

# # The //2 is necessary since `profile` counts multiply and accumulate
# # as two flops, here we report the total number of multiply accumulate ops
# flops = graph_info.total_float_ops // 2
# print('Flops: {:,}'.format(flops))